In [1]:
from bs4 import BeautifulSoup
import requests
from json import loads
import csv
import os
import pandas as pd
import numpy as np

# Constants
SETS = [
    ['SOR', 252], 
    ['SHD', 262], 
    ['TWI', 257], 
    ['JTL', 262], 
    ['LOF', 264], 
    ['IBH', 104], 
    ['SEC', 264],
]

In [2]:
'''
download_cards(sets: list)

Description: Accesses swu-db api and returns a dataframe with all of the card
information from the listed sets.

Param:
    sets: list
        List of lists of SWU set codes and number of cards to download.
        Appropriate Sets: 'SOR', 'SHD', 'TWI', 'JTL', 'LOF', 'IBH', 'SEC'
        See constant `SETS` above. Includes the standard length of each set.
Returns:
    pandas.DataFrame
        Headers: 'Set', 'Number', 'Name', 'Subtitle', 'Type', 'Aspects', 'Traits',
                 'Arenas', 'Cost', 'Power', 'HP', 'FrontText', 'EpicAction',
                 'DoubleSided', 'BackArt', 'BackText', 'Rarity', 'Unique',
                 'Keywords', 'Artist', 'VariantType', 'MarketPrice', 'LowPrice',
                 'FrontArt', 'FoilPrice', 'LowFoilPrice'
'''
def download_cards(sets: list):
    cards = []
    print('Downloading Card Information')
    for set_, set_len in sets:
        for num in range(set_len):
            c = loads(requests.get(f'https://api.swu-db.com/cards/{set_}/{num + 1:03}').text)
            if 'message' in c.keys()
            cards.append(c)
            print(f'Completed: {set_}-{num + 1:03}', end='\r')
    cards = pd.DataFrame(cards).set_index(['Set','Number'])
    return cards

In [3]:
def download_all_cards():
    cards = download_cards(SETS)
    cards.to_csv('cards.csv', index=True)

In [4]:
def download_set_cards(set: list):
    set = [set] if type(set[0]) != type([]) else set
    cards = download_cards(set)
    return cards

In [5]:
def append_set_cards(set: list):
    cards = pd.read_csv('cards.csv').set_index(['Set', 'Number'])
    set = [set] if type(set[0]) != type([]) else set
    cards = pd.concat([cards, download_set_cards(set)])
    return cards

In [6]:
def get_cards_from_csv():
    return pd.read_csv('cards.csv').set_index(['Set','Number'])

In [7]:
c = get_cards_from_csv()

In [8]:
c[c['Name']=='Shadowed Undercity']

,,Name,Subtitle,Type,Aspects,Traits,Arenas,Cost,Power,HP,FrontText,...,Keywords,Artist,VariantType,MarketPrice,LowPrice,FrontArt,FoilPrice,LowFoilPrice,message,error
Set,Number,,,,,,,,,,,,,,,,,,,,,


In [9]:
c.iloc['LOF'].iloc['20']

Name                Subtitle     Type  \
Set Number                                                             
SOR 1.0            Director Krennic   Aspiring to Authority   Leader   
    2.0                 Iden Versio  Infero Squad Commander   Leader   
    3.0                   Chewbacca          Walking Carpet   Leader   
    4.0                Chirrut Îmwe      One With the Force   Leader   
    5.0              Luke Skywalker         Faithful Friend   Leader   
...                             ...                     ...      ...   
SEC 261.0         Inspiring Senator                     NaN     Unit   
    262.0           Ando Commission                     NaN     Unit   
    263.0            Assassin Probe                     NaN     Unit   
    264.0   Clandestine Connections                     NaN  Upgrade   
    265.0      Chancellor Palpatine        How Liberty Dies   Leader   

                              Aspects                      Traits      Arenas  \
Set Number                                                                      
SOR 1.0     ['Vigilance', 'Villainy']    ['IMPERIAL', 'OFFICIAL']  ['Ground']   
    2.0     ['Vigilance', 'Villainy']     ['IMPERIAL', 'TROOPER']  ['Ground']   
    3.0      ['Vigilance', 'Heroism']   ['UNDERWORLD', 'WOOKIEE']  ['Ground']   
    4.0      ['Vigilance', 'Heroism']          ['FORCE', 'REBEL']  ['Ground']   
    5.0      ['Vigilance', 'Heroism']          ['FORCE', 'REBEL']  ['Ground']   
...                               ...                         ...         ...   
SEC 261.0                          []    ['REPUBLIC', 'OFFICIAL']  ['Ground']   
    262.0                          []  ['SEPARATIST', 'OFFICIAL']  ['Ground']   
    263.0                          []     ['SEPARATIST', 'DROID']  ['Ground']   
    264.0                          []                  ['SUPPLY']         NaN   
    265.0   ['Vigilance', 'Villainy']    ['REPUBLIC', 'OFFICIAL']  ['Ground']   

            Cost  Power   HP  \
Set Number                     
SOR 1.0      5.0    2.0  7.0   
    2.0      6.0    4.0  4.0   
    3.0      7.0    2.0  9.0   
    4.0      5.0    3.0  5.0   
    5.0      6.0    4.0  7.0   
...          ...    ...  ...   
SEC 261.0    3.0    3.0  3.0   
    262.0    4.0    4.0  3.0   
    263.0    5.0    4.0  4.0   
    264.0    2.0    NaN  NaN   
    265.0    7.0    6.0  8.0   

                                                    FrontText  ...  \
Set Number                                                     ...   
SOR 1.0                Each friendly damaged unit gets +1/+0.  ...   
    2.0     Action [{Exhaust}]: If an enemy unit was defea...  ...   
    3.0     Action [{Exhaust}]: Play a unit that costs 3 o...  ...   
    4.0     Action [{{Exhaust}}]: Give a unit +0/+2 for th...  ...   
    5.0     Action [{C=1}, {Exhaust}]: Give a Shield token...  ...   
...                                                       ...  ...   
SEC 261.0   When Defeated: The next Official unit you play...  ...   
    262.0   Sentinel (Enemy units in this arena must attac...  ...   
    263.0   When Defeated: Deal 1 damage to each exhausted...  ...   
    264.0   Attached unit gains: “On Attack: You may pay 2...  ...   
    265.0   Action [C=1, Exhaust]: Search the top 5 cards ...  ...   

                        Keywords             Artist VariantType MarketPrice  \
Set Number                                                                    
SOR 1.0              ['Restore']         Ario Murti      Normal        0.09   
    2.0             ['Shielded']        Amélie Hutt      Normal        0.17   
    3.0     ['Grit', 'Sentinel']     Eslam Aboshady      Normal        0.10   
    4.0                      NaN  Sandra Chlewińska      Normal        0.11   
    5.0                      NaN      Borja Pindado      Normal        2.20   
...                          ...                ...         ...         ...   
SEC 261.0                    NaN   Kaihound Studios      Normal        0.30   
 

In [66]:
set_, num = ['LOF', 20]
c = loads(requests.get(f'https://api.swu-db.com/cards/{set_}/{num:03}').text)
if 'message' in c.keys():
    html = requests.get(f'https://www.swu-db.com/card/{set_}/{num:03}').text
    soup = BeautifulSoup(html, 'html.parser')
    c['Set'] = set_
    c['Number'] = num
    c['Name'], c['Subtitle'] = soup.find(id='card-title-aspects').get_text().strip().split(' - ')
    c['Type'] = soup.find(class_='card-type-span').get_text().strip().title()
    c['Aspects'] = [soup.find(alt='Aspect Icon')['src'].split('/')[-1].split('.')[0].title()]
    c['FrontArt'] = soup.find(alt='Card Image')['src']
    c['FrontText'] = soup.find(class_='card-text').get_text().strip()
    for item in soup.find_all(class_='card-set-item'):
        if item.find(class_='card-set-title'):
            c[item.find(class_='card-set-title').get_text().strip()] = item.find(class_='card-set-text').get_text().strip()
    for key in c.keys():
        if key not in ['Set', 'Number', 'Name', 'Subtitle', 'Type', 'Aspects', 'Traits', 'Arenas', 'Cost', 'Power', 'HP', 'FrontText', 'EpicAction', ]
else:
    None
c

{'message': 'Internal server error',
 'Set': 'Legends of the Force',
 'Number': 20,
 'Name': 'Nightsister Lair',
 'Subtitle': 'Dathomir',
 'Type': 'Base',
 'Aspects': ['Vigilance'],
 'FrontArt': 'https://cdn.swu-db.com/images/cards/LOF/020.png',
 'FrontText': 'When a friendly Force unit attacks: The Force is with you (create your Force token).',
 'HP': '28',
 'Card Number': '020/264',
 'Rarity': 'Common',
 'Artist': 'Adrien Girod',
 'Variant': 'Original'}

In [67]:
c = loads(requests.get(f'https://api.swu-db.com/cards/SOR/30').text)
c

{'Set': 'SOR',
 'Number': '030',
 'Name': 'Chopper Base',
 'Subtitle': 'Atollon',
 'Type': 'Base',
 'Aspects': ['Cunning'],
 'HP': '30',
 'DoubleSided': False,
 'Rarity': 'Common',
 'Unique': False,
 'Artist': 'Tyler Edlin',
 'VariantType': 'Normal',
 'MarketPrice': '0.15',
 'LowPrice': '0.05',
 'FrontArt': 'https://cdn.swu-db.com/images/cards/SOR/030.png'}

In [121]:
def get_from_html(set, num):
    c = {}
    html = requests.get(f'https://www.swu-db.com/card/{set_}/{num:03}').text
    soup = BeautifulSoup(html, 'html.parser')
    c['Set'] = set_
    c['Number'] = num
    c['Name'], c['Subtitle'] = soup.find(id='card-title-aspects').get_text().strip().split(' - ')
    c['Unique'] = True if '⟡' in c['Name'] else False
    c['Name'] = c['Name'].replace('⟡ ', '')
    c['Type'] = soup.find(class_='card-type-span').get_text().strip().title()
    c['Traits'] = [a.get_text() for a in soup.find(class_='card-traits').find_all('a')]
    c['Traits'] = c['Traits'] if len(c['Traits']) > 0 else None
    c['Aspects'] = [a['src'].split('/')[-1].split('.')[0].title() for a in soup.find_all(alt='Aspect Icon')]
    c['FrontArt'] = soup.find(alt='Card Image')['src']
    c['FrontText'] = soup.find(class_='card-text').get_text().strip()
    c['EpicAction'] = soup.find(class_='epic-action-text').get_text().strip() if soup.find(class_='epic-action-text') else None
    for item in soup.find_all(class_='card-set-item'):
        if item.find(class_='card-set-title'):
            if '/' in item.find(class_='card-set-title').get_text().strip():
                k1, k2 = item.find(class_='card-set-title').get_text().strip().split('/')
                v1, v2 = item.find(class_='card-set-text').get_text().strip().split('/')
                for k, v in zip([k1, k2], [v1, v2]):
                    c[k] = v
            else:
                c[item.find(class_='card-set-title').get_text().strip()] = item.find(class_='card-set-text').get_text().strip()
    rem = []
    for key in c.keys():
        if key not in ['Set','Number','Name','Subtitle','Type','Aspects','Traits','Arenas','Cost','Power','HP','FrontText','EpicAction','DoubleSided','BackArt','BackText','Rarity','Unique','Keywords','Artist','VariantType','MarketPrice','LowPrice','FrontArt','FoilPrice','LowFoilPrice']:
            rem.append(key)
    for key in rem:
        c.pop(key)
    return c

In [122]:
c = get_from_html('LOF', 20)
c

{'Set': 'Legends of the Force',
 'Number': 20,
 'Name': 'Nightsister Lair',
 'Subtitle': 'Dathomir',
 'Unique': False,
 'Type': 'Base',
 'Traits': None,
 'Aspects': ['Vigilance'],
 'FrontArt': 'https://cdn.swu-db.com/images/cards/LOF/020.png',
 'FrontText': 'When a friendly Force unit attacks: The Force is with you (create your Force token).',
 'EpicAction': None,
 'HP': '28',
 'Rarity': 'Common',
 'Artist': 'Adrien Girod'}

In [116]:
for key in ['Set','Number','Name','Subtitle','Type','Aspects','Traits','Arenas','Cost','Power','HP','FrontText','EpicAction','DoubleSided','BackArt','BackText','Rarity','Unique','Keywords','Artist','VariantType','MarketPrice','LowPrice','FrontArt','FoilPrice','LowFoilPrice']:
    if key not in c.keys():
        print(key)

Arenas
EpicAction
DoubleSided
BackArt
BackText
Keywords
VariantType
MarketPrice
LowPrice
FoilPrice
LowFoilPrice
